# ML2025F: 2025-2 머신러닝 프로젝트
## 문제 정의
#### 연구 가설
인종과 부모 교육 수준이 학생의 학업 수준에 영향을 줄 것이다  
#### 머신러닝 모델
시각화: 피처별 학업 수준 플롯 작성  
머신러닝: 피처로부터 학업 수준을 예측하는 회귀모델 작성  

## 데이터 정의
#### 데이터 확보 방안
Kaggle 오픈 데이터 “Exploring Student Achievement”  
#### 데이터 명세 기술
파일 포맷: .csv  
데이터 사이즈: 컬럼 8개, 레코드 1000개  
컬럼 명세  
성별: 남, 여  
인종: A, B, C, D, E  
부모 교육 수준: 고교 중퇴, 고교 졸업, 대학 중퇴, 대학 수료, 대학 졸업(학사), 대학 졸업(석사)  
급식 유형: 표준 급식, 무료 급식  
시험 준비 상태: 완료, 완료하지 않음  
수학 점수  
읽기 점수  
쓰기 점수  

#### 데이터 로드

In [101]:
import numpy as np
import pandas as pd

In [102]:
df=pd.read_csv("./data.csv")

# 컬럼 리네임
df=df.rename(
    columns={
        "gender":                      "GEN",
        "race/ethnicity":              "ETH",
        "parental level of education": "EDU",
        "lunch":                       "LUN",
        "test preparation course":     "TPC",
        "math score":                  "MAT",
        "reading score":               "REA",
        "writing score":               "WRI",
    }
)
df.head()

,GEN,ETH,EDU,LUN,TPC,MAT,REA,WRI
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## 데이터 처리 과정
#### 누락 데이터 처리 과정

In [103]:
# 결측치 탐지
print(df.isnull().sum())

GEN    0
ETH    0
EDU    0
LUN    0
TPC    0
MAT    0
REA    0
WRI    0
dtype: int64


#### 이상치 처리 과정

In [104]:
# 점수 피처 범위 확인
print(((df[["MAT"]]>=0) & (df[["MAT"]]<=100)).sum())
print(((df[["REA"]]>=0) & (df[["REA"]]<=100)).sum())
print(((df[["WRI"]]>=0) & (df[["WRI"]]<=100)).sum())

MAT    1000
dtype: int64
REA    1000
dtype: int64
WRI    1000
dtype: int64


#### 레이블 인코딩

In [105]:
# 피처 레이블 도메인 확인
for i in df["GEN"].unique(): print(i) # gender
for i in df["ETH"].unique(): print(i) # race/ethnicity
for i in df["EDU"].unique(): print(i) # parental level of education
for i in df["LUN"].unique(): print(i) # lunch
for i in df["TPC"].unique(): print(i) # test preparation course

female
male
group B
group C
group A
group D
group E
bachelor's degree
some college
master's degree
associate's degree
high school
some high school
standard
free/reduced
none
completed


In [106]:
# 원-핫 인코딩: 성별
df["GEN_M"]=(df["GEN"]=="male")
df["GEN_F"]=(df["GEN"]=="female")
df=df.drop(columns=["GEN"])

# 원-핫 인코딩: 인종
df["ETH_A"]=(df["ETH"]=="group A")
df["ETH_B"]=(df["ETH"]=="group B")
df["ETH_C"]=(df["ETH"]=="group C")
df["ETH_D"]=(df["ETH"]=="group D")
df["ETH_E"]=(df["ETH"]=="group E")
df=df.drop(columns=["ETH"])

df.head()

,EDU,LUN,TPC,MAT,REA,WRI,GEN_M,GEN_F,ETH_A,ETH_B,ETH_C,ETH_D,ETH_E
0,bachelor's degree,standard,none,72,72,74,False,True,False,True,False,False,False
1,some college,standard,completed,69,90,88,False,True,False,False,True,False,False
2,master's degree,standard,none,90,95,93,False,True,False,True,False,False,False
3,associate's degree,free/reduced,none,47,57,44,True,False,True,False,False,False,False
4,some college,standard,none,76,78,75,True,False,False,False,True,False,False


In [107]:
# 레이블 인코딩: 부모 교육 수준, 급식 유형, 시험 준비 상태
MAP_EDU={
    "some high school":  1,
    "high school":       2,
    "associate's degree":3,
    "some college":      4,
    "bachelor's degree": 5,
    "master's degree":   6,
}
MAP_LUN={
    "standard":    1,
    "free/reduced":2,
}
MAP_TPC={
    "none":     1,
    "completed":2,
}
df["EDU"]=df["EDU"].map(MAP_EDU)
df["LUN"]=df["LUN"].map(MAP_LUN)
df["TPC"]=df["TPC"].map(MAP_TPC)

In [108]:
# 컬럼 순서 조정

df=df.reindex(
    columns=[
        "GEN_M","GEN_F",
        "ETH_A","ETH_B","ETH_C","ETH_D","ETH_E",
        "EDU",
        "LUN",
        "TPC",
        "MAT",
        "REA",
        "WRI",
    ]
)
df.head()

,GEN_M,GEN_F,ETH_A,ETH_B,ETH_C,ETH_D,ETH_E,EDU,LUN,TPC,MAT,REA,WRI
0,False,True,False,True,False,False,False,5,1,1,72,72,74
1,False,True,False,False,True,False,False,4,1,2,69,90,88
2,False,True,False,True,False,False,False,6,1,1,90,95,93
3,True,False,True,False,False,False,False,3,2,1,47,57,44
4,True,False,False,False,True,False,False,4,1,1,76,78,75


## 모델 튜닝 과정
#### 파라미터 튜닝 및 성능 변화 기술

#### 다른 모델과의 비교 등

## 결론
#### 연구 문제에 대한 결론 도출